## How to Grid Search Hyperparameters for Deep Learning Models in Python with Keras

- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [1]:
# Use scikit-learn to grid search the weight initialization
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

In [2]:
# Function to create model, required for KerasClassifier
def create_model(init_mode='uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # return value
    return model

In [3]:
# fix random seed for reproducibility
tf.random.set_seed(seed=1234)

In [4]:
# load dataset
dataset = np.loadtxt("dataset/diabetes.csv", delimiter=",")

In [5]:
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
# show to dataset
pd.concat([
    pd.DataFrame(X, columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']),
    pd.DataFrame(Y, columns=['Y'])
], axis=1)

,x1,x2,x3,x4,x5,x6,x7,x8,Y
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,0.0
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,0.0
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,0.0
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,1.0


In [7]:
# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=1)

In [8]:
model

KerasClassifier(
	model=<function create_model at 0x000002D8FECEF310>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=10
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	class_weight=None
)

In [9]:
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

In [10]:
param_grid = dict(model__init_mode=init_mode)

In [11]:
param_grid

{'model__init_mode': ['uniform',
  'lecun_uniform',
  'normal',
  'zero',
  'glorot_normal',
  'glorot_uniform',
  'he_normal',
  'he_uniform']}

In [12]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [13]:
grid_result = grid.fit(X, Y)

Epoch 1/100
77/77 [==============================] - 2s 3ms/step - loss: 0.6661 - accuracy: 0.6510
Epoch 2/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6466 - accuracy: 0.6510
Epoch 3/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6326 - accuracy: 0.6536
Epoch 4/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6226 - accuracy: 0.6641
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6168 - accuracy: 0.6641
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6151 - accuracy: 0.6810
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6049 - accuracy: 0.6797
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6012 - accuracy: 0.6940
Epoch 9/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6021 - accuracy: 0.6862
Epoch 10/100
77/77 [==============================] - 0s 2ms/step - loss: 0.5973 - accuracy: 0.7018
Epoch 11/

In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.726562 using {'model__init_mode': 'uniform'}


In [15]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.726562 (0.028348) with: {'model__init_mode': 'uniform'}
0.687500 (0.022326) with: {'model__init_mode': 'lecun_uniform'}
0.690104 (0.031466) with: {'model__init_mode': 'normal'}
0.651042 (0.001841) with: {'model__init_mode': 'zero'}
0.696615 (0.025976) with: {'model__init_mode': 'glorot_normal'}
0.709635 (0.015733) with: {'model__init_mode': 'glorot_uniform'}
0.627604 (0.080771) with: {'model__init_mode': 'he_normal'}
0.679688 (0.017758) with: {'model__init_mode': 'he_uniform'}
